# Notebook de Carga de Datos iniciales

Este notebook esta destinado a la carga inicial de los datos en los distintas base de datos

## Importación de librerias

In [42]:
from pathlib import Path
from db_connections import client,db_neo4j,db_redis
import os
from src import mongo
from neo4j import GraphDatabase
from src import neo4j


## Conexión con bases de datos

In [43]:
# =====================
# TEST DE CONEXIONES
# =====================
print("MongoDB colecciones:", client["clase"].list_collection_names())
print("Neo4j driver activo:", db_neo4j is not None)
print("Redis ping:", db_redis.ping())

MongoDB colecciones: ['libros', 'usuarios', 'hoteles', 'reservas', 'destinos', 'actividades', 'alumnos']
Neo4j driver activo: True
Redis ping: True


## Carga de Datos

In [44]:
# --- MONGO ---
nombre_base="clase"
nombre_colecciones = ["usuarios","destinos","hoteles","reservas","actividades"]
nombre_archivos= ["usuarios.csv","destinos.csv","hoteles.csv","reservas.csv","actividades.csv"]

for i in range (5):
    ruta=Path("fuentes") / nombre_archivos[i]
    mongo.crear_coleccion(nombre_base,nombre_colecciones[i],recrear=True)
    mongo.cargar_csv_a_coleccion(ruta,nombre_base,nombre_colecciones[i])

✅ Se insertaron 1000 documentos en 'usuarios'.
✅ Se insertaron 50 documentos en 'destinos'.
✅ Se insertaron 100 documentos en 'hoteles'.
✅ Se insertaron 200 documentos en 'reservas'.
✅ Se insertaron 120 documentos en 'actividades'.


##### Contar la cantidad de documentos existentes en cada colección

In [45]:
for i in range (5):
    cantidad=mongo.contar_documentos(nombre_base,nombre_colecciones[i])
    print(f"La cantidad de documentos de la coleccion {nombre_colecciones[i]} es {cantidad}")

La cantidad de documentos de la coleccion usuarios es 1000
La cantidad de documentos de la coleccion destinos es 50
La cantidad de documentos de la coleccion hoteles es 100
La cantidad de documentos de la coleccion reservas es 200
La cantidad de documentos de la coleccion actividades es 120


##### Listar datos de las colecciones

In [46]:
# Lista el primer dato de cada colección como prueba de la carga

for i in range (5):
    cursor=mongo.obtener_cursor(nombre_base,nombre_colecciones[i], 1,proyeccion={"_id":0})
    print(f"Colección {nombre_colecciones[i]}:")
    mongo.imprimir_cursor(cursor)

    

Colección usuarios:
{'apellido': 'Aguiló',
 'email': 'jose-ignaciocalatayud@gmail.com',
 'nombre': 'Brígida',
 'telefono': '+34 701 95 78 02',
 'usuario_id': 1}
Colección destinos:
{'ciudad': 'Valencia',
 'destino_id': 1,
 'pais': 'Brasil',
 'precio_promedio': 175291,
 'tipo': 'Playa'}
Colección hoteles:
{'calificacion': 4,
 'ciudad': 'Guadalajara',
 'hotel_id': 1,
 'nombre': 'Fábrica JJH S.Coop.',
 'precio': 218005,
 'servicios': 'pileta;wifi;spa'}
Colección reservas:
{'destino_id': 15,
 'estado': 'Cancelada',
 'fecha_reserva': '2025-08-07',
 'precio_total': 59359,
 'reserva_id': 1,
 'usuario_id': 319}
Colección actividades:
{'actividad_id': 1,
 'ciudad': 'Valladolid',
 'nombre': 'Organic object-oriented Internet solution',
 'precio': 78337,
 'tipo': 'aventura'}


## CREO USUARIOS EN NEO4J

In [47]:
# IMPORTO DATOS USUARIO DESDE MONGO
usuarios = mongo.obtener_cursor(
    "clase",
    nombre_coleccion="usuarios",
    proyeccion={"_id":0,"usuario_id": 1, "nombre": 1, "apellido": 1}
)
usuarios=list(usuarios)

# CREO NODO EN NEO4J





In [48]:

# Traer datos de MongoDB
destinos = mongo.obtener_cursor(
    "clase",
    nombre_coleccion="destinos",
    proyeccion={"_id": 0, "destino_id": 1, "ciudad": 1, "pais": 1}
)
destinos = list(destinos)

# Conexión a Neo4j
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD", "neo4j123")
driver = GraphDatabase.driver("bolt://neo4j:7687", auth=("neo4j", NEO4J_PASSWORD))

# Crear nodos en Neo4j
with driver.session() as session:
    for destino in destinos:
        nodo = session.execute_write(neo4j.crear_nodo, "Destino", "destino_id", destino)
        print(f"Nodo creado o encontrado: {nodo['n']}")


Nodo creado o encontrado: <Node element_id='4:f0b1a310-9db2-4083-8af1-ac27d102df9d:1' labels=frozenset({'Destino'}) properties={'ciudad': 'Valencia', 'destino_id': 1, 'pais': 'Brasil'}>
Nodo creado o encontrado: <Node element_id='4:f0b1a310-9db2-4083-8af1-ac27d102df9d:2' labels=frozenset({'Destino'}) properties={'ciudad': 'Guadalajara', 'destino_id': 2, 'pais': 'Brasil'}>
Nodo creado o encontrado: <Node element_id='4:f0b1a310-9db2-4083-8af1-ac27d102df9d:3' labels=frozenset({'Destino'}) properties={'ciudad': 'Tarragona', 'destino_id': 3, 'pais': 'Chile'}>
Nodo creado o encontrado: <Node element_id='4:f0b1a310-9db2-4083-8af1-ac27d102df9d:4' labels=frozenset({'Destino'}) properties={'ciudad': 'Murcia', 'destino_id': 4, 'pais': 'Brasil'}>
Nodo creado o encontrado: <Node element_id='4:f0b1a310-9db2-4083-8af1-ac27d102df9d:5' labels=frozenset({'Destino'}) properties={'ciudad': 'Guadalajara', 'destino_id': 5, 'pais': 'Italia'}>
Nodo creado o encontrado: <Node element_id='4:f0b1a310-9db2-4083-8